 
# Team 63 Exploratory Data Analysis : Demographic County Data
## Preliminary

 



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Info about the Texas Demographic Data
df_demo=pd.read_csv('../data/cc-est2019-alldata-48.csv')
print('Demographic: ', df_demo.shape, df_demo.columns)


In [ ]:
df_demo.head()

In [ ]:
df_demo.tail()

In [ ]:
#according to the County Characteristics, year 1 is the 2010 Census Population and 
# years 2&3 is estimates base and estimate for 2010,
# years 4-12 represent Population estimates for years 2011 - 2019.  
# I'm going to drop years 2 and 3 and change the entry to the the year of estimation
notneeded = df_demo[(df_demo['YEAR']==2) | (df_demo['YEAR']==3)].index
df_demo.drop(notneeded, inplace=True)
convert_year_nums = {'YEAR': {1:2010, 4:2011, 5:2012, 6:2013, 7:2014, 8:2015, 
                             9:2016, 10:2017, 11:2018, 12:2019}}
df_demo.replace(convert_year_nums, inplace=True)
df_demo['YEAR'].unique()


In [ ]:
#Number of counties in the dataset 
print('Total number of Texas Counties in dataset: ', df_demo.groupby('COUNTY').ngroups)

In [ ]:
#Our group is not interested in all counties, but a specific list of counties.
#Since the column for COUNTY and state reference a County Code, we will all a County Name column 
#with a data set of Texas Counties as found on the State Education website
#load the counties csv
TX_counties = pd.read_csv('../data/Counties.csv')
TX_counties.head()
#pull out only the country code and its associated name
County_Names = TX_counties.loc[:, 'FIPS':'FENAME']
# Merge the df_demo with the added columns from County_Name on FIPS ... County Code
df_demo_counties = pd.merge(left=df_demo, right=County_Names, left_on='COUNTY', right_on='FIPS')
# Since only have shutdown information from only a few of the 254 counties in TX
# We only want to look at the information for those counties
spec_counties = ['BELL','BEXAR','BOWIE','BRAZORIA','BRAZOS','HARRIS','CASTRO',
                 'CHAMBERS','COLLIN','Comal','DALLAS','DENTON','ELLIS','EL PASO']
df_spec_counties = df_demo_counties[df_demo_counties['FENAME'].isin(spec_counties)].reset_index(drop=True)
#df_spec_counties = df_spec_counties.rename(columns={'FENAME': 'County Name'})



In [ ]:
print('Total number of Special Texas Counties in dataset: ', df_spec_counties.groupby('CTYNAME').ngroups)
df_spec_counties['CTYNAME'].unique()

In [ ]:
#Group the rows by County and Year, then sum the columns so I don't have to look at the data by age group
grouped_spec_counties = df_spec_counties.groupby(['FENAME', 'YEAR'])
df=grouped_spec_counties.sum()
df = df.reset_index()


In [ ]:
df.head()

In [ ]:
# Population Breakdown per County
df.groupby(['FENAME']).TOT_POP.mean().sort_values(ascending=False)

In [ ]:
#since population is broken up in terms of male and female for ethnicity groups
#I want to add a total value per ethnicity groups
df['BA_All'] = df['BA_FEMALE']+df['BA_MALE'] # black only
df['WA_All'] = df['WA_FEMALE']+df['WA_MALE'] # white only
df['H_All'] = df['H_FEMALE']+df['H_MALE']    # hispanic
df['AA_All'] = df['AA_FEMALE']+df['AA_MALE'] # asian only
df['IAC_All'] = df['IAC_FEMALE']+df['IAC_MALE']# american indian or alaska native in combo
df['HBA_All'] = df['HBA_FEMALE']+df['HBA_MALE'] #hispanic or black 
df['NHBA_All'] = df['NHBA_FEMALE']+df['NHBA_MALE'] #not hispanic, black or combination

In [ ]:
#minority breakdown Black Only, White Only, Hispanic Only, Asian Only, Indigeinous Only mean over the years 
#sorted by highest Black Only population
df.groupby(['FENAME'])['BA_All', 'WA_All', 'H_All', 'AA_All', 'IAC_All'].mean().sort_values(by='BA_All', ascending=False)

In [ ]:
#Non-Hispanic White and Hispanic & Black Breakdowns mean over the years
#sorted by highest Hispanic and Black population
df.groupby(['FENAME'])['NHBA_All', 'HBA_All'].mean().sort_values(by='HBA_All', ascending=False)

In [ ]:
#highest percentage of Hispanic and Black residents on average over the years
df['HBARatio']=df['HBA_All']/df['TOT_POP']
df.groupby(['FENAME'])['HBARatio'].mean().sort_values(ascending=False)

In [ ]:
grp=df.groupby(['FENAME'])
legendnames = grp.apply(lambda x: x.FENAME)
plots = grp.plot('YEAR', 'HBARatio', legend=True, ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
thecounties = list(df.FENAME.unique())
for c in thecounties: 
    df1 = df[df['FENAME']== c]
    df1.plot(x='YEAR', y='HBARatio', kind='line', ax=ax, label=c)


In [ ]:
#total Population in the counties
df.groupby(['YEAR','FENAME'])['TOT_POP'].sum().unstack().plot()

In [ ]:
#Look at the mean of the population of the counties
df.groupby(['FENAME'])['TOT_POP'].mean().plot(kind='barh')

In [ ]:

#Add Percentages Columns since the County Populations vary significantly
df['BlackOnly_Percentage'] = df['BA_All'] /df['TOT_POP']
df['WhiteOnly_Percentage'] =  df['WA_All'] /df['TOT_POP']
df['IndigenousOnly_Percentage'] =  df['IAC_All'] /df['TOT_POP']
df['HispanicsOnly_Percentage'] =  df['H_All'] /df['TOT_POP']
df['AsianOnly_Percentage'] =  df['AA_All'] /df['TOT_POP']
df['HispanicsandorBlack_Percentage'] = df['HBA_All'] /df['TOT_POP']
df[df['YEAR']==2019].describe()





In [ ]:
df.groupby(['FENAME'])['HispanicsandorBlack_Percentage'].mean().plot(kind='barh')

In [ ]:
# Black Only Percentages in County
fig, ax = plt.subplots(figsize=(10,8))
thecounties = list(df.FENAME.unique())
for c in thecounties: 
    df1 = df[df['FENAME']== c]
    df1.plot(x='YEAR', y='BlackOnly_Percentage', kind='line', ax=ax, label=c)
  

In [ ]:
# Hispanics Only Percentages in County
fig, ax = plt.subplots(figsize=(10,8))
thecounties = list(df.FENAME.unique())
for c in thecounties: 
    df1 = df[df['FENAME']== c]
    df1.plot(x='YEAR', y='HispanicsOnly_Percentage', kind='line', ax=ax, label=c)

In [ ]:
# Hispanic and Blacks Percentages in County
fig, ax = plt.subplots(figsize=(10,8))
thecounties = list(df.FENAME.unique())
for c in thecounties: 
    df1 = df[df['FENAME']== c]
    df1.plot(x='YEAR', y='HispanicsandorBlack_Percentage', kind='line', ax=ax, label=c)

In [ ]:
# White Only Percentages in County
fig, ax = plt.subplots(figsize=(10,8))
thecounties = list(df.FENAME.unique())
for c in thecounties: 
    df1 = df[df['FENAME']== c]
    df1.plot(x='YEAR', y='WhiteOnly_Percentage', kind='line', ax=ax, label=c)

In [ ]:
df.groupby(['FENAME'])['WhiteOnly_Percentage'].mean().plot(kind='barh')

Based on the analysis, the white only percentage of the counties are decreasing (significantly in Collin, Brazoria and Bexar County), while slightly in others. 
* Castro County has the highest percentage of White only residents, while Bell has the lowest percentage
* Bell has the highest percentage of Hispanic and Black residents, while Bowie has the lowest percentage